# Métrica de Levenshtein y Damerau-Levenshtein

In [1]:
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import statsmodels.api as sm
import os
from re import sub

In [6]:
def memoize(f):
    cache = {}

    def memoizedFunction(*args):
        if args not in cache:
            cache[args] = f(*args)
        return cache[args]

    memoizedFunction.cache = cache
    return memoizedFunction

## Métrica de Levenshtein 

In [13]:
@memoize
def dist1(word1, word2):
    if not word1 or not word2:
        return max(len(word1), len(word2))
    elif word1[-1] == word2[-1]:
        return dist(word1[:-1], word2[:-1])
    else:
        return min(dist(word1[:-1], word2) + 1,
                        dist(word1, word2[:-1]) + 1,
                        dist(word1[:-1], word2[:-1]) + 1)

levenshteinDist = dist1

## Métrica de Damerau-Levenshtein 

In [14]:
@memoize
def dist2(word1, word2):
    if not word1 or not word2:
        return max(len(word1), len(word2))
    elif word1[-1] == word2[-1]:
        return dist2(word1[:-1], word2[:-1])
    else:
        minDist = min(dist2(word1[:-1], word2) + 1,
                                dist2(word1, word2[:-1]) + 1,
                                dist2(word1[:-1], word2[:-1]) + 1)

        # transposiciones
        if len(word1) > 1 and len(word2) > 1:
            if word1[-2] == word2[-1]:
                transposedWord1 = word1[:-2] + word1[-1] + word1[-2]
                minDist = min(minDist, dist2(transposedWord1[:-1], word2))

            if word2[-2] == word1[-1]:
                transposedWord2 = word2[:-2] + word2[-1] + word2[-2]
                minDist = min(minDist, dist2(word1, transposedWord2[:-1]))

        return minDist

damerauLevenshteinDist = dist2

In [15]:
damerauLevenshteinDist('agua','atl')

3

In [16]:
damerauLevenshteinDist('xolo','mozo')

2

In [17]:
damerauLevenshteinDist('transposición','trasnposición')

1

In [18]:
levenshteinDist('transposición','trasnposición')

2

## Cuentos Nahuatl-Español 

In [19]:
os.path.join("","")

''

In [11]:
def getCuentos():
    cuentosEsp = []
    cuentosNah = []
    folder = "corpusCuentos"
    files = [os.path.join(folder,val) for val in os.listdir(folder) if os.path.isfile(os.path.join(folder,val))]
    for file in files:
        f = open(file,"r",encoding="ISO-8859-1")
        l = f.read().split("\n")
        if file.split(os.path.sep)[1][0:3]=="esp":
            cuentosEsp.append([x for x in l if bool(x)])
        elif file.split(os.path.sep)[1][0:3]=="nah":
            cuentosNah.append([x for x in l if bool(x)])
        f.close()
    return cuentosEsp, cuentosNah

In [12]:
def getSentences(cuentos):
    f = []
    for cuento in cuentos:
        c = []
        for par in cuento:
            sentences = par.split(".")
            sentences = [sub(r"[^\w\s]", " ",sen) for sen in sentences if bool(sen)]
            c.extend(sentences)
        f.append(c)
    return f

In [13]:
cuentosEsp, cuentosNah = getCuentos()

In [14]:
cuentosEsp, cuentosNah = getSentences(cuentosEsp), getSentences(cuentosNah)

Obtenemos las palabras en ambos idiomas

In [15]:
palabras_esp=[]
oracionesEsp = [item.split(" ") for l in cuentosEsp for item in l]
p_esp = [item for lis in oracionesEsp for item in lis]
p_esp=set(p_esp)
palabras_esp=list(p_esp)
palabras_esp.pop(0)

''

In [16]:
palabras_nah=[]
oracionesNah = [item.split(" ") for l in cuentosNah for item in l]
p_nah = [item for lis in oracionesNah for item in lis]
p_nah=set(p_nah)
palabras_nah=list(p_nah)
palabras_nah.pop(0)

''

## Red de Damerau-Levenshtein 

Incluimos los vértices aislados i.e. damerauLevenshteinDist(i,j) > 1

In [17]:
def levenshteinNetwork(corpus1,corpus2):
    levenG = nx.Graph()
    nodes = corpus1 + corpus2
    levenG.add_nodes_from(nodes)
    for i in corpus1:
        for j in corpus2:
            if dist2(i,j) == 1:
                levenG.add_edge(i,j)
    return levenG

NO incluimos los vértices aislados i.e. damerauLevenshteinDist(i,j) > 1

In [18]:
def levenshteinNetwork2(corpus1,corpus2):
    levenG = nx.Graph()
    nodes = corpus1 + corpus2
    levenG.add_nodes_from(nodes)
    for i in corpus1:
        for j in corpus2:
            if dist2(i,j) == 1:
                levenG.add_edge(i,j)
    list(nx.isolates(levenG))
    levenG.remove_nodes_from(list(nx.isolates(levenG)))
    return levenG

In [19]:
grafica1 = levenshteinNetwork(palabras_esp,palabras_nah)
type(grafica1)

networkx.classes.graph.Graph

In [20]:
nx.info(grafica1)

'Name: \nType: Graph\nNumber of nodes: 1518\nNumber of edges: 227\nAverage degree:   0.2991'

In [196]:
grafica2 = levenshteinNetwork2(palabras_esp,palabras_nah)

In [197]:
nx.info(grafica2)

'Name: \nType: Graph\nNumber of nodes: 155\nNumber of edges: 227\nAverage degree:   2.9290'

In [189]:
nx.write_gexf(grafica2,'grafica2.gexf',version='1.2draft')

Segundo ejemplo damerauLevenshteinDist(i,j) > 2

In [199]:
def levenshteinNetwork3(corpus1,corpus2):
    levenG = nx.Graph()
    nodes = corpus1 + corpus2
    levenG.add_nodes_from(nodes)
    for i in corpus1:
        for j in corpus2:
            if dist2(i,j) <= 2:
                levenG.add_edge(i,j)
    list(nx.isolates(levenG))
    levenG.remove_nodes_from(list(nx.isolates(levenG)))
    return levenG

In [200]:
grafica3 = levenshteinNetwork3(palabras_esp,palabras_nah)

In [201]:
nx.info(grafica3)

'Name: \nType: Graph\nNumber of nodes: 466\nNumber of edges: 2539\nAverage degree:  10.8970'

In [202]:
nx.write_gexf(grafica3,'grafica3.gexf',version='1.2draft')